In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt 

In [4]:
import os

In [5]:
import cv2

In [6]:
import tensorflow as tf

In [7]:
Datadirectory = "./Dataset 2/train"

In [8]:
Classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

In [9]:
training_data = []
img_size = 224

def create_training_data():
    for category in Classes:
        path = os.path.join(Datadirectory, category);
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
             img_array = cv2.imread(os.path.join(path, img))
             new_array = cv2.resize(img_array, (img_size, img_size))
             training_data.append([new_array, class_num])
            except Exception as e:
               pass

In [10]:
create_training_data()

In [11]:
print(len(training_data))

28709


In [12]:
import random

random.shuffle(training_data)

In [13]:
x = []
y = []

for features,label in training_data:
    x.append(features)
    y.append(label)

In [14]:
x = np.array(x).reshape(-1, img_size, img_size, 3)

In [15]:
# Normalize the data
for i in range(0,len(training_data)):
    x[i] = x[i]/255.0

In [16]:
y = np.array(y)

In [17]:
y.shape

(28709,)

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [19]:
model = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 2s 0us/step


In [20]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [21]:
base_input = model.layers[0].input

In [22]:
base_output = model.layers[-2].output

In [23]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [25]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)

In [26]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_3')>

In [27]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [28]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [29]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
from tensorflow import keras

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='emotion_recogntion_model_checkpoint.h5',
    save_weights_only=True,
    save_best_only=True,
    save_freq='epoch'
)

In [34]:
new_model.fit(x, y, epochs=15, callbacks=[checkpoint_callback])

Epoch 1/15
898/898 [==============================] - 1791s 2s/step - loss: 1.7940 - accuracy: 0.2611
Epoch 2/15
898/898 [==============================] - 2019s 2s/step - loss: 1.7943 - accuracy: 0.2603
Epoch 3/15
898/898 [==============================] - 1897s 2s/step - loss: 1.7936 - accuracy: 0.2611
Epoch 4/15
898/898 [==============================] - 2582s 3s/step - loss: 1.7939 - accuracy: 0.2598
Epoch 5/15
898/898 [==============================] - 2008s 2s/step - loss: 1.7934 - accuracy: 0.2601
Epoch 6/15
652/898 [====================>.........] - ETA: 9:35 - loss: 1.7938 - accuracy: 0.2599

KeyboardInterrupt: 